In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libs

In [2]:
import numpy as np
import pandas as pd
import copy
import sys
import cv2
import random
from scipy import ndimage
from scipy.spatial import distance
import time
import math
import pickle
import os

In [3]:
!pip install pyprind
import pyprind

In [4]:
!pip install pygame
from os import environ
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'
import pygame
from pygame.locals import KEYDOWN, K_q

     |████████████████████████████████| 18.3 MB 1.2 MB/s 


In [5]:
import time
import itertools
import random
from pprint import pprint
import io

## Darp Class

In [6]:
np.set_printoptions(threshold=sys.maxsize)


class DARP():
    def __init__(self, nx, ny, MaxIter, CCvariation, randomLevel, dcells, importance, notEqualPortions, initial_positions, portions, obstacles_positions, visualization):
        self.rows = nx
        self.cols = ny
        self.visualization = visualization
        empty_space = []
        if nx > ny:
            for j in range(ny, nx):
                for i in range(nx):
                    empty_space.append((i, j))
            self.cols = self.rows
        elif ny > nx:
            for j in range(nx, ny):
                for i in range(ny):
                    empty_space.append((j, i))
            self.rows = self.cols

        self.GridEnv = np.full(shape=(self.rows, self.cols), fill_value=0)

        for cell in empty_space:
            self.GridEnv[cell[0], cell[1]] = 1

        if obstacles_positions != []:
            for obstacle in obstacles_positions:
                self.GridEnv[obstacle[0], obstacle[1]] = 1
        for pos in initial_positions:
            self.GridEnv[pos[0], pos[1]] = 2

        #prints the Grid Layout with bots and obstalces
        #print("Given Grid area:")
        #print(self.GridEnv)

        self.droneNo = 0
        self.A = np.zeros((self.rows, self.cols))
        self.init_robot_pos = []
        self.ob = 0
        self.defineRobotsObstacles()
        self.MaxIter = MaxIter
        self.CCvariation = CCvariation
        self.randomLevel = randomLevel
        self.dcells = dcells
        self.importance = importance
        self.notEqualPortions = notEqualPortions
        self.connectivity = np.zeros((self.droneNo, self.rows, self.cols))
        self.BinaryRobotRegions = np.zeros((self.droneNo, self.rows, self.cols), dtype=bool)

        # If user has not defined custom portions divide area equally for all drones
        self.Rportions = np.zeros((self.droneNo))
        if (not notEqualPortions):
            for i in range(self.droneNo):
                self.Rportions[i] = 1.0/self.droneNo
        else:
            for i in range(self.droneNo):
                self.Rportions[i] = portions[i]

        self.AllDistances, self.termThr, self.Notiles, self.DesireableAssign, self.TilesImportance, self.MinimumImportance, self.MaximumImportance= self.construct_Assignment_Matrix()
        self.MetricMatrix = copy.deepcopy(self.AllDistances)
        self.BWlist = np.zeros((self.droneNo, self.rows, self.cols))
        self.ArrayOfElements = np.zeros(self.droneNo)
        self.color = []

        for r in range(self.droneNo):
            self.color.append(list(np.random.choice(range(256), size=3)))

        if self.visualization:
            self.assignment_matrix_visualization = darp_area_visualization(self.A, self.droneNo, self.color)

        self.success = self.update()

    def defineRobotsObstacles(self):
        for i in range(self.rows):
            for j in range(self.cols):
                if self.GridEnv[i, j] == 2:
                    self.GridEnv[i, j] = self.droneNo
                    self.A[i, j] = self.droneNo
                    self.droneNo += 1
                    self.init_robot_pos.append((i, j))
                elif(self.GridEnv[i, j] == 1):
                    self.ob += 1
                    self.GridEnv[i, j] = -2
                else:
                    self.GridEnv[i, j] = -1

    def update(self):
        success = False
        cancelled = False
        criterionMatrix = np.zeros((self.rows, self.cols))

        while self.termThr <= self.dcells and not success and not cancelled:
            downThres = (self.Notiles - self.termThr*(self.droneNo-1))/(self.Notiles*self.droneNo)
            upperThres = (self.Notiles + self.termThr)/(self.Notiles*self.droneNo)

            success = True

            #main optimization loop
            iteration = 0

            while iteration <= self.MaxIter and not cancelled:
                self.assign()
                ConnectedMultiplierList = np.ones((self.droneNo, self.rows, self.cols))
                ConnectedRobotRegions = np.zeros(self.droneNo)
                plainErrors = np.zeros((self.droneNo))
                divFairError = np.zeros((self.droneNo))

                for r in range(self.droneNo):
                    ConnectedMultiplier = np.ones((self.rows, self.cols))
                    ConnectedRobotRegions[r] = True
                    self.update_connectivity()
                    image = np.uint8(self.connectivity[r, :, :])
                    num_labels, labels_im = cv2.connectedComponents(image, connectivity=4)
                    if num_labels > 2:
                        ConnectedRobotRegions[r] = False
                        BinaryRobot, BinaryNonRobot = self.constructBinaryImages(labels_im, r)
                        ConnectedMultiplier = self.CalcConnectedMultiplier(
                            self.NormalizedEuclideanDistanceBinary(True, BinaryRobot, BinaryNonRobot),
                            self.NormalizedEuclideanDistanceBinary(False, BinaryRobot, BinaryNonRobot))
                    ConnectedMultiplierList[r, :, :] = ConnectedMultiplier
                    plainErrors[r] = self.ArrayOfElements[r]/(self.DesireableAssign[r]*self.droneNo)
                    if plainErrors[r] < downThres:
                        divFairError[r] = downThres - plainErrors[r]
                    elif plainErrors[r] > upperThres:
                        divFairError[r] = upperThres - plainErrors[r]

                #Prints the Grid Assignment for each bot
                if self.IsThisAGoalState(self.termThr, ConnectedRobotRegions):
                    #print("\nFinal Assignment Matrix:")
                    #print(self.A)
                    break

                TotalNegPerc = 0
                totalNegPlainErrors = 0
                correctionMult = np.zeros(self.droneNo)

                for r in range(self.droneNo):
                    if divFairError[r] < 0:
                        TotalNegPerc += np.absolute(divFairError[r])
                        totalNegPlainErrors += plainErrors[r]

                    correctionMult[r] = 1

                for r in range(self.droneNo):
                    if totalNegPlainErrors != 0:
                        if divFairError[r] < 0:
                            correctionMult[r] = 1 + (plainErrors[r]/totalNegPlainErrors)*(TotalNegPerc/2)
                        else:
                            correctionMult[r] = 1 - (plainErrors[r]/totalNegPlainErrors)*(TotalNegPerc/2)

                        criterionMatrix = self.calculateCriterionMatrix(
                                self.TilesImportance[r],
                                self.MinimumImportance[r],
                                self.MaximumImportance[r],
                                correctionMult[r],
                                divFairError[r] < 0)

                    self.MetricMatrix[r] = self.FinalUpdateOnMetricMatrix(
                            criterionMatrix,
                            self.generateRandomMatrix(),
                            self.MetricMatrix[r],
                            ConnectedMultiplierList[r, :, :])

                iteration += 1
                if self.visualization:
                    self.assignment_matrix_visualization.placeCells(self.A)
                # time.sleep(0.5)

            if iteration >= self.MaxIter:
                self.MaxIter = self.MaxIter/2
                success = False
                self.termThr += 1

        self.getBinaryRobotRegions()
        return success

    def getBinaryRobotRegions(self):

        for i in range(self.rows):
            for j in range(self.cols):
                if self.A[i][j] < self.droneNo:
                    self.BinaryRobotRegions[int(self.A[i, j]), i, j] = True
                # else:
                #     self.BinaryRobotRegions[int(self.A[i, j]), i, j] = False

    def generateRandomMatrix(self):
        RandomMa = np.zeros((self.rows, self.cols))
        randomlevel = 0.0001
        for i in range(self.rows):
            for j in range(self.cols):
                RandomMa[i][j] = 2*randomlevel*random.uniform(0, 1) + (1 - randomlevel)

        return RandomMa

    def FinalUpdateOnMetricMatrix(self, CM, RM, currentOne, CC):
        MMnew = np.zeros((self.rows, self.cols))

        for i in range(self.rows):
            for j in range(self.cols):
                MMnew[i, j] = currentOne[i, j]*CM[i, j]*RM[i, j]*CC[i, j]

        return MMnew

    def IsThisAGoalState(self, thresh, connectedRobotRegions):
        for r in range(self.droneNo):
            #TODO if thresh == 0?
            if np.absolute(self.DesireableAssign[r] - self.ArrayOfElements[r]) > thresh or not connectedRobotRegions[r]:
                return False
        return True

    def update_connectivity(self):
        for i in range(self.droneNo):
            for j in range(self.rows):
                for k in range(self.cols):
                    if self.A[j, k] == i:
                        self.connectivity[i, j, k] = 255
                    else:
                        self.connectivity[i, j, k] = 0

    def constructBinaryImages(self, A, val):
        BinaryRobot = np.copy(A)
        BinaryNonRobot = np.copy(A)
        for i in range(self.rows):
            for j in range(self.cols):
                if A[i, j] == 1:
                    BinaryRobot[i, j] = 1
                    BinaryNonRobot[i, j] = 0
                elif A[i, j] != 0:
                    BinaryRobot[i, j] = 0
                    BinaryNonRobot[i, j] = 1

        return BinaryRobot, BinaryNonRobot

    def assign(self):
        self.BWlist = np.zeros((self.droneNo, self.rows, self.cols))
        for r in range(self.droneNo):
            self.BWlist[r, self.init_robot_pos[r][0], self.init_robot_pos[r][1]] = 1

        self.ArrayOfElements = np.zeros(self.droneNo)
        for i in range(self.rows):
            for j in range(self.cols):
                if self.GridEnv[i, j] == -1:
                    minV = self.MetricMatrix[0, i, j]
                    indMin = 0
                    for r in range(self.droneNo):
                        if self.MetricMatrix[r, i, j] < minV:
                            minV = self.MetricMatrix[r, i, j]
                            indMin = r

                    self.A[i][j] = indMin
                    self.BWlist[indMin, i, j] = 1
                    self.ArrayOfElements[indMin] += 1

                elif self.GridEnv[i, j] == -2:
                    self.A[i, j] = self.droneNo

    # Construct Assignment Matrix
    def construct_Assignment_Matrix(self):
        Notiles = self.rows*self.cols
        fair_division = 1/self.droneNo
        effectiveSize = Notiles - self.droneNo - self.ob
        termThr = 0

        if effectiveSize % self.droneNo != 0:
            termThr = 1

        DesireableAssign = np.zeros(self.droneNo)
        MaximunDist = np.zeros(self.droneNo)
        MaximumImportance = np.zeros(self.droneNo)
        MinimumImportance = np.zeros(self.droneNo)

        for i in range(self.droneNo):
            DesireableAssign[i] = effectiveSize * self.Rportions[i]
            MinimumImportance[i] = sys.float_info.max
            if (DesireableAssign[i] != int(DesireableAssign[i]) and termThr != 1):
                termThr = 1

        AllDistances = np.zeros((self.droneNo, self.rows, self.cols))
        TilesImportance = np.zeros((self.droneNo, self.rows, self.cols))

        for x in range(self.rows):
            for y in range(self.cols):
                tempSum = 0
                for r in range(self.droneNo):
                    AllDistances[r, x, y] = np.linalg.norm(np.array(self.init_robot_pos[r]) - np.array((x, y)))  # E!
                    if AllDistances[r, x, y] > MaximunDist[r]:
                        MaximunDist[r] = AllDistances[r, x, y]
                    tempSum += AllDistances[r, x, y]

                for r in range(self.droneNo):
                    if tempSum - AllDistances[r, x, y] != 0:
                        TilesImportance[r, x, y] = 1/(tempSum - AllDistances[r, x, y])
                    else:
                        TilesImportance[r, x, y] = 1
                    # Todo FixMe!
                    if TilesImportance[r, x, y] > MaximumImportance[r]:
                        MaximumImportance[r] = TilesImportance[r, x, y]

                    if TilesImportance[r, x, y] < MinimumImportance[r]:
                        MinimumImportance[r] = TilesImportance[r, x, y]

        return AllDistances, termThr, Notiles, DesireableAssign, TilesImportance, MinimumImportance, MaximumImportance

    def calculateCriterionMatrix(self, TilesImportance, MinimumImportance, MaximumImportance, correctionMult, smallerthan0,):
        returnCrit = np.zeros((self.rows, self.cols))
        for i in range(self.rows):
            for j in range(self.cols):
                if self.importance:
                    if smallerthan0:
                        returnCrit[i, j] = (TilesImportance[i, j] - MinimumImportance)*((correctionMult-1)/(MaximumImportance-MinimumImportance)) + 1
                    else:
                        returnCrit[i, j] = (TilesImportance[i, j] - MinimumImportance)*((1-correctionMult)/(MaximumImportance-MinimumImportance)) + correctionMult
                else:
                    returnCrit[i, j] = correctionMult

        return returnCrit

    def CalcConnectedMultiplier(self, dist1, dist2):
        returnM = np.zeros((self.rows, self.cols))
        MaxV = 0
        MinV = sys.float_info.max

        for i in range(self.rows):
            for j in range(self.cols):
                returnM[i, j] = dist1[i, j] - dist2[i, j]
                if MaxV < returnM[i, j]:
                    MaxV = returnM[i, j]
                if MinV > returnM[i, j]:
                    MinV = returnM[i, j]

        for i in range(self.rows):
            for j in range(self.cols):
                returnM[i, j] = (returnM[i, j]-MinV)*((2*self.CCvariation)/(MaxV - MinV)) + (1-self.CCvariation)

        return returnM

    def NormalizedEuclideanDistanceBinary(self, RobotR, BinaryRobot, BinaryNonRobot):
        if RobotR:
            distRobot = ndimage.morphology.distance_transform_edt(np.logical_not(BinaryRobot))
        else:
            distRobot = ndimage.morphology.distance_transform_edt(np.logical_not(BinaryNonRobot))

        MaxV = np.max(distRobot)
        MinV = np.min(distRobot)

        #Normalization
        for i in range(self.rows):
            for j in range(self.cols):
                if RobotR:
                    distRobot[i, j] = (distRobot[i, j] - MinV)*(1/(MaxV-MinV)) + 1
                else:
                    distRobot[i, j] = (distRobot[i, j] - MinV)*(1/(MaxV-MinV))

        return distRobot

## Edges, Graphs and Turns

In [7]:
class Edge(object):
    def __init__(self, _from, to, weight):
        self.src = _from
        self.dst = to
        self.weight = weight

    def __eq__(self, other):
        return (self.src == other.src and self.dst == other.dst and self.weight == other. weight)

    def __hash__(self):
        return hash((self.src, self.dst, self.weight))


class Graph:

    def __init__(self, nodes, arg_edgelist):
        self.nodes = nodes
        self.num_nodes = len(self.nodes)
        self.edgelist = arg_edgelist
        self.parent = []
        self.rank = []
        # mst stores edges of the minimum spanning tree
        self.mst = []

    def FindParent(self, node):
        # With path-compression.

        if node != self.parent[node]:
            self.parent[node] = self.FindParent(self.parent[node])
        return self.parent[node]

        # Without path compression
        # if node == self.parent[node] :
        #    return node
        # return self.FindParent(self.parent[node])

    def KruskalMST(self):

        # Sort objects of an Edge class based on attribute (weight)
        self.edgelist.sort(key=lambda Edge: Edge.weight)

        self.parent = [None] * self.num_nodes
        self.rank = [None] * self.num_nodes

        for n in self.nodes:
            self.parent[n] = n  # Every node is the parent of itself at the beginning
            self.rank[n] = 0   # Rank of every node is 0 at the beginning

        for edge in self.edgelist:
            root1 = self.FindParent(edge.src)
            root2 = self.FindParent(edge.dst)

            # Parents of the source and destination nodes are not in the same subset
            # Add the edge to the spanning tree
            if root1 != root2:
                self.mst.append(edge)
                if self.rank[root1] < self.rank[root2]:
                    self.parent[root1] = root2
                    self.rank[root2] += 1
                else:
                    self.parent[root2] = root1
                    self.rank[root1] += 1

        # print("\nEdges of minimum spanning tree in graph :", end=' ')
        cost = 0
        for edge in self.mst:
            # print("[" + str(edge.src) + "-" + str(edge.dst) + "](" + str(edge.weight) + ")", end=' ')
            cost += edge.weight
        # print("\nCost of minimum spanning tree : "+str(cost))

      
class turns:
  
    def __init__(self, paths):
        """
        paths: List of lists of moves per drone
        """
        self.paths = paths
        self.turns = []

    def __str__(self):
        return (
            '\n'
            f'Turns: {self.turns}\n'
            f'Average: {self.avg:.3f}\n'
            f'Standard Deviation: {self.std:.3f}\n')

    def count_turns(self):
        for path in self.paths:
            num_turns = -1
            last_move = ""
            for move in path:
                if move[0] == move[2]:
                    current_move = "horizontal"
                elif move[1] == move[3]:
                    current_move = "vertical"

                if last_move != current_move:
                    num_turns += 1

                last_move = current_move
            self.turns.append(num_turns)

        self.avg = np.average(self.turns)
        self.std = np.std(self.turns)


## Kruskal

In [8]:
class Kruskal(object):
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.allEdges = []
        self.MAX_NODES = self.rows*self.cols
        self.nodes = {}
        for node in range(self.MAX_NODES):
            self.nodes[node] = None
        self.mst = []

    def initializeGraph(self, A, connect4, mode):
        cost1 = 1
        cost2 = 1

        for i in range(self.rows):
            for j in range(self.cols):
                if (A[i][j]):
                    if (mode == 0):
                        cost2 = self.rows - i
                    elif mode == 1:
                        cost2 = i+1
                    elif mode == 2:
                        cost1 = self.cols - j
                    elif mode == 3:
                        cost1 = j+1

                    if (i > 0 and A[i-1][j]):
                        self.AddToAllEdges(i*self.cols+j, (i-1)*self.cols+j, cost1)
                    if (i < self.rows-1 and A[i+1][j]):
                        self.AddToAllEdges(i*self.cols+j, (i+1)*self.cols+j, cost1)
                    if (j > 0 and A[i][j-1]):
                        self.AddToAllEdges(i*self.cols+j, i*self.cols+j-1, cost2)
                    if (j < self.cols-1 and A[i][j+1]):
                        self.AddToAllEdges(i*self.cols+j, i*self.cols+j+1, cost2)

                    if not connect4:
                        if (i > 0 and j > 0 and A[i-1][j-1]):
                            AddToAllEdges(i*self.cols+j, (i-1)*self.cols+j-1, 1)
                        if (i < rows-1 and j < self.cols-1 and A[i+1][j+1]):
                            AddToAllEdges(i*self.cols+j, (i+1)*self.cols+j+1, 1)
                        if (i > rows-1 and j > 0 and A[i+1][j-1]):
                            AddToAllEdges(i*self.cols+j, (i+1)*self.cols+j-1, 1)
                        if (i > 0 and j < self.cols-1 and A[i-1][j+1]):
                            AddToAllEdges(i*self.cols+j, (i-1)*self.cols+j+1, 1)

    def AddToAllEdges(self, _from: int, to: int, cost):
        self.allEdges.insert(0, Edge(_from, to, cost))

        if (self.nodes[_from]) is None:
            self.nodes[_from] = set()
            self.nodes[_from].add(_from)

        if (self.nodes[to]) is None:
            self.nodes[to] = set()
            self.nodes[to].add(to)

        return

    def performKruskal(self):
        g1 = Graph(self.nodes, self.allEdges)
        g1.KruskalMST()
        self.mst = g1.mst

## Calculate Trajectories

In [9]:
class CalculateTrajectories():
    def __init__(self, r, c, MST):
        self.MAX_NODES = 4*r*c
        self.PathSequence = []
        self.rows = r
        self.cols = c
        self.MSTvector = MST
        self.MSTedges = len(self.MSTvector)
        self.allEdges = set()
        self.nodes = {}
        for node in range(self.MAX_NODES):
            self.nodes[node] = None

    def initializeGraph(self, A, connect4):
        for i in range(2*self.rows):
            for j in range(2*self.cols):
                if A[i, j]:
                    if i > 0 and A[i-1][j]:
                        self.AddToAllEdges(i*2*self.cols+j, (i-1)*2*self.cols+j, 1)
                    if i < 2*self.rows-1 and A[i+1][j]:
                        self.AddToAllEdges(i*2*self.cols+j, (i+1)*2*self.cols+j, 1)
                    if j > 0 and A[i][j-1]:
                        self.AddToAllEdges(i*2*self.cols+j, i*2*self.cols+j-1, 1)
                    if j < 2*self.cols-1 and A[i][j+1]:
                        self.AddToAllEdges(i*2*self.cols+j, i*2*self.cols+j+1, 1)

                    if not connect4:
                        if i > 0 and j > 0 and A[i-1][j-1]:
                            self.AddToAllEdges(i*2*self.cols+j, (i-1)*2*self.cols+j-1, 1)
                        if i < 2*self.rows-1 and j < 2*self.cols-1 and A[i+1][j+1]:
                            self.AddToAllEdges(i*2*self.cols+j, (i+1)*2*self.cols+j+1, 1)
                        if i > 2*self.rows-1 and j > 0 and A[i+1][j-1]:
                            self.AddToAllEdges(i*2*self.cols+j, (i+1)*2*self.cols+j-1, 1)
                        if i > 0 and j < 2*self.cols-1 and A[i-1][j+1]:
                            self.AddToAllEdges(i*2*self.cols+j, (i-1)*2*self.cols+j+1, 1)

    def AddToAllEdges(self, _from: int, to: int, cost):
        self.allEdges.add(Edge(_from, to, cost))

        if (self.nodes[_from]) is None:
            self.nodes[_from] = set()

        self.nodes[_from].add(to)

        if (self.nodes[to]) is None:
            self.nodes[to] = set()

        self.nodes[to].add(_from)

    def RemoveTheAppropriateEdges(self):
        for i in range(self.MSTedges):
            e = self.MSTvector[i]
            maxN = max(e.src, e.dst)
            minN = min(e.src, e.dst)

            if np.absolute(e.src - e.dst) == 1:
                alpha = (4*minN+3) - 2*(maxN % self.cols)
                eToRemove = Edge(alpha, alpha+2*self.cols, 1)
                eToRemoveMirr = Edge(alpha+2*self.cols, alpha, 1)
                eToRemove2 = Edge(alpha+1, alpha+1+2*self.cols, 1)
                eToRemove2Mirr = Edge(alpha+1+2*self.cols, alpha+1, 1)

            else:
                alpha = (4*minN+2*self.cols) - 2*(maxN % self.cols)
                eToRemove = Edge(alpha, alpha+1, 1)
                eToRemoveMirr = Edge(alpha+1, alpha, 1)
                eToRemove2 = Edge(alpha+2*self.cols, alpha+1+2*self.cols, 1)
                eToRemove2Mirr = Edge(alpha+1+2*self.cols, alpha+2*self.cols, 1)

            if eToRemove in self.allEdges:
                self.SafeRemoveEdge(eToRemove)

            if eToRemoveMirr in self.allEdges:
                self.SafeRemoveEdge(eToRemoveMirr)

            if eToRemove2 in self.allEdges:
                self.SafeRemoveEdge(eToRemove2)

            if eToRemove2Mirr in self.allEdges:
                self.SafeRemoveEdge(eToRemove2Mirr)

    def SafeRemoveEdge(self, curEdge):
        try:
            self.allEdges.remove(curEdge)
            # successful removal from priority queue: allEdges
            if curEdge.dst in self.nodes[curEdge.src]:
                self.nodes[curEdge.src].remove(curEdge.dst)
            if curEdge.src in self.nodes[curEdge.dst]:
                self.nodes[curEdge.dst].remove(curEdge.src)

        except KeyError:
            # This is a serious problem
            print("TreeSet should have contained this element!!")
            sys.exit(1)

    def CalculatePathsSequence(self, StartingNode):

        currentNode = StartingNode
        RemovedNodes = set()
        movement = []
        PathSequence = []

        movement.append(2*self.cols)
        movement.append(-1)
        movement.append(-2*self.cols)
        movement.append(1)

        found = False
        prevNode = 0
        for idx in range(4):
            if (currentNode + movement[idx]) in list(self.nodes[currentNode]):
                prevNode = currentNode + movement[idx]
                found = True
                break

        if not found:
            return

        while True:
            if currentNode != StartingNode:
                RemovedNodes.add(currentNode)

            offset = movement.index(prevNode-currentNode)

            prevNode = currentNode

            found = False
            for idx in range(4):
                if (prevNode+movement[(idx+offset) % 4] in self.nodes[prevNode]) and not (prevNode+movement[(idx+offset) % 4] in RemovedNodes):

                    currentNode = prevNode + movement[(idx+offset) % 4]
                    found = True
                    break

            if not found:
                return

            if (prevNode in self.nodes[currentNode]):
                self.nodes[currentNode].remove(prevNode)

            if (currentNode in self.nodes[prevNode]):
                self.nodes[prevNode].remove(currentNode)

            i = int(currentNode/(2*self.cols))
            j = currentNode % (2*self.cols)
            previ = int(prevNode/(2*self.cols))
            prevj = prevNode % (2*self.cols)
            self.PathSequence.append((previ, prevj, i, j))


## Visualization

In [10]:
# CONSTANTS:
SCREENSIZE = WIDTH, HEIGHT = 800, 800
BLACK = (0, 0, 0)
GREY = (160, 160, 160)


class visualize_paths():
    def __init__(self, AllRealPaths, subCellsAssignment, DroneNo, color):
        self.AllRealPaths = AllRealPaths
        self.subCellsAssignment = subCellsAssignment
        self.DroneNo = DroneNo
        self._VARS = {'surf': False,
                      'gridWH': 800,
                      'gridOrigin': (0, 0),
                      'gridCells': self.subCellsAssignment.shape[0],
                      'lineWidth': 2}
        self.color = color

    def visualize_paths(self, mode):
        pygame.init()
        self._VARS['surf'] = pygame.display.set_mode(SCREENSIZE)
        pygame.display.set_caption('Mode: ' + str(mode))
        while True:
            keep_going = self.checkEvents()
            if not keep_going:
                break
            self._VARS['surf'].fill(GREY)
            self.drawSquareGrid(self._VARS['gridOrigin'],
                                self._VARS['gridWH'],
                                self._VARS['gridCells'])
            self.placeCells()
            pygame.display.update()

    def visualize_darp_area(self):
        pygame.init()
        self._VARS['surf'] = pygame.display.set_mode(SCREENSIZE)
        while True:
            keep_going = self.checkEvents()
            if not keep_going:
                break
            self._VARS['surf'].fill(GREY)
            self.drawSquareGrid(self._VARS['gridOrigin'],
                                self._VARS['gridWH'],
                                self._VARS['gridCells'])
            self.darp_area()
            pygame.display.update()

    def darp_area(self):
        cellBorder = 1
        celldimX = celldimY = (self._VARS['gridWH']/self._VARS['gridCells']) - (cellBorder*2)
        for row in range(self.subCellsAssignment.shape[0]):
            for column in range(self.subCellsAssignment.shape[1]):
                if (self.subCellsAssignment[column][row] == self.DroneNo):
                    self.drawSquareCell(
                        self._VARS['gridOrigin'][0] + (celldimY*row)
                        + cellBorder + (2*row*cellBorder) + self._VARS['lineWidth']/2,
                        self._VARS['gridOrigin'][1] + (celldimX*column)
                        + cellBorder + (2*column*cellBorder) + self._VARS['lineWidth']/2,
                        celldimX, celldimY, BLACK)
                    continue
                for r in range(self.DroneNo):
                    if(self.subCellsAssignment[column][row] == r):
                        self.drawSquareCell(
                            self._VARS['gridOrigin'][0] + (celldimY*row)
                            + cellBorder + (2*row*cellBorder) + self._VARS['lineWidth']/2,
                            self._VARS['gridOrigin'][1] + (celldimX*column)
                            + cellBorder + (2*column*cellBorder) + self._VARS['lineWidth']/2,
                            celldimX, celldimY, self.color[r])

        pygame.display.update()

    def placeCells(self):
        cellBorder = 0
        celldimX = celldimY = (self._VARS['gridWH']/self._VARS['gridCells']) - (cellBorder*2)
        for r in range(self.DroneNo):
            for point in self.AllRealPaths[r]:
                # size = (300, 300)
                color = pygame.Color(255, 0, 0)
                pygame.draw.line(self._VARS['surf'],
                                 self.color[r],
                                 (self._VARS['gridOrigin'][0] + (celldimX*point[1] + celldimX/2),
                                  self._VARS['gridOrigin'][1] + (celldimY*point[0]) + celldimY/2),
                                 (self._VARS['gridOrigin'][0] + (celldimX*point[3]) + celldimX/2,
                                  self._VARS['gridOrigin'][1] + (celldimY*point[2]) + celldimY/2), width=4)

        cellBorder = 1
        celldimX = celldimY = (self._VARS['gridWH']/self._VARS['gridCells']) - (cellBorder*2)
        for row in range(self.subCellsAssignment.shape[0]):
            for column in range(self.subCellsAssignment.shape[1]):
                if (self.subCellsAssignment[column][row] == self.DroneNo):
                    self.drawSquareCell(
                        self._VARS['gridOrigin'][0] + (celldimY*row)
                        + 1 + (2*row*1) + self._VARS['lineWidth']/2,
                        self._VARS['gridOrigin'][1] + (celldimX*column)
                        + 1 + (2*column*1) + self._VARS['lineWidth']/2,
                        celldimX, celldimY, BLACK)

        pygame.display.update()

    # Draw filled rectangle at coordinates
    def drawSquareCell(self, x, y, dimX, dimY, color):
        pygame.draw.rect(
         self._VARS['surf'], color,
         (x, y, dimX, dimY)
        )

    def drawSquareGrid(self, origin, gridWH, cells):

        CONTAINER_WIDTH_HEIGHT = gridWH
        cont_x, cont_y = origin

        # DRAW Grid Border:
        # TOP lEFT TO RIGHT
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (cont_x, cont_y),
          (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y), self._VARS['lineWidth'])
        # # BOTTOM lEFT TO RIGHT
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (cont_x, CONTAINER_WIDTH_HEIGHT + cont_y),
          (CONTAINER_WIDTH_HEIGHT + cont_x,
           CONTAINER_WIDTH_HEIGHT + cont_y), self._VARS['lineWidth'])
        # # LEFT TOP TO BOTTOM
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (cont_x, cont_y),
          (cont_x, cont_y + CONTAINER_WIDTH_HEIGHT), self._VARS['lineWidth'])
        # # RIGHT TOP TO BOTTOM
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y),
          (CONTAINER_WIDTH_HEIGHT + cont_x,
           CONTAINER_WIDTH_HEIGHT + cont_y), self._VARS['lineWidth'])

        # Get cell size, just one since its a square grid.
        cellSize = CONTAINER_WIDTH_HEIGHT/cells

        for x in range(cells):
            pygame.draw.line(
               self._VARS['surf'], BLACK,
               (cont_x + (cellSize * x), cont_y),
               (cont_x + (cellSize * x), CONTAINER_WIDTH_HEIGHT + cont_y), 2)
        # # HORIZONTAl DIVISIONS
            pygame.draw.line(
              self._VARS['surf'], BLACK,
              (cont_x, cont_y + (cellSize*x)),
              (cont_x + CONTAINER_WIDTH_HEIGHT, cont_y + (cellSize*x)), 2)

    def checkEvents(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == KEYDOWN and event.key == K_q):
                pygame.quit()
                return False
        return True


class darp_area_visualization(object):
    def __init__(self, Assignment_matrix, DroneNo, color):
        self.Assignment_matrix = Assignment_matrix
        self.DroneNo = DroneNo
        self._VARS = {'surf': False,
                      'gridWH': 800,
                      'gridOrigin': (0, 0),
                      'gridCells': self.Assignment_matrix.shape[0],
                      'lineWidth': 2}
        self.color = color
        pygame.init()
        self._VARS['surf'] = pygame.display.set_mode(SCREENSIZE)
        self.checkEvents()
        self._VARS['surf'].fill(GREY)
        self.drawSquareGrid(
         self._VARS['gridOrigin'], self._VARS['gridWH'], self._VARS['gridCells'])
        self.placeCells(self.Assignment_matrix)
        pygame.display.set_caption('Assignment Matrix')
        pygame.display.update()
        # time.sleep(5)

    def checkEvents(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif event.type == KEYDOWN and event.key == K_q:
                pygame.quit()
                sys.exit()

    def drawSquareGrid(self, origin, gridWH, cells):

        CONTAINER_WIDTH_HEIGHT = gridWH
        cont_x, cont_y = origin

        # DRAW Grid Border:
        # TOP lEFT TO RIGHT
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (cont_x, cont_y),
          (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y), self._VARS['lineWidth'])
        # # BOTTOM lEFT TO RIGHT
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (cont_x, CONTAINER_WIDTH_HEIGHT + cont_y),
          (CONTAINER_WIDTH_HEIGHT + cont_x,
           CONTAINER_WIDTH_HEIGHT + cont_y), self._VARS['lineWidth'])
        # # LEFT TOP TO BOTTOM
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (cont_x, cont_y),
          (cont_x, cont_y + CONTAINER_WIDTH_HEIGHT), self._VARS['lineWidth'])
        # # RIGHT TOP TO BOTTOM
        pygame.draw.line(
          self._VARS['surf'], BLACK,
          (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y),
          (CONTAINER_WIDTH_HEIGHT + cont_x,
           CONTAINER_WIDTH_HEIGHT + cont_y), self._VARS['lineWidth'])

        # Get cell size, just one since its a square grid.
        cellSize = CONTAINER_WIDTH_HEIGHT/cells

        for x in range(cells):
            pygame.draw.line(
               self._VARS['surf'], BLACK,
               (cont_x + (cellSize * x), cont_y),
               (cont_x + (cellSize * x), CONTAINER_WIDTH_HEIGHT + cont_y), 2)
        # # HORIZONTAl DIVISIONS
            pygame.draw.line(
              self._VARS['surf'], BLACK,
              (cont_x, cont_y + (cellSize*x)),
              (cont_x + CONTAINER_WIDTH_HEIGHT, cont_y + (cellSize*x)), 2)

    def placeCells(self, Assignment_matrix):
        cellBorder = 1
        celldimX = celldimY = (self._VARS['gridWH']/self._VARS['gridCells']) - (cellBorder*2)
        for row in range(self.Assignment_matrix.shape[0]):
            for column in range(self.Assignment_matrix.shape[1]):
                if (self.Assignment_matrix[column][row] == self.DroneNo):
                    self.drawSquareCell(
                        self._VARS['gridOrigin'][0] + (celldimY*row)
                        + cellBorder + (2*row*cellBorder) + self._VARS['lineWidth']/2,
                        self._VARS['gridOrigin'][1] + (celldimX*column)
                        + cellBorder + (2*column*cellBorder) + self._VARS['lineWidth']/2,
                        celldimX, celldimY, BLACK)
                    continue
                for r in range(self.DroneNo):
                    if(self.Assignment_matrix[column][row] == r):
                        self.drawSquareCell(
                            self._VARS['gridOrigin'][0] + (celldimY*row)
                            + cellBorder + (2*row*cellBorder) + self._VARS['lineWidth']/2,
                            self._VARS['gridOrigin'][1] + (celldimX*column)
                            + cellBorder + (2*column*cellBorder) + self._VARS['lineWidth']/2,
                            celldimX, celldimY, self.color[r])

        pygame.display.update()

    def drawSquareCell(self, x, y, dimX, dimY, color):
        pygame.draw.rect(
         self._VARS['surf'], color,
         (x, y, dimX, dimY)
        )

## Main

In [11]:
class DARPinPoly(DARP):
    def __init__(self, nx, ny, MaxIter, CCvariation, randomLevel, dcells, importance, notEqualPortions, initial_positions, portions, obstacles_positions, visualization):
        DARP.__init__(self, nx, ny, MaxIter, CCvariation, randomLevel, dcells, importance, notEqualPortions, initial_positions, portions, obstacles_positions, visualization)

        final_local = {}
        final_local['NA'] = 0
        if not self.success:
            final_local['NA'] = 1
            exit()
            #sys.exit(0)

        mode_to_drone_turns = dict()
        FinalPaths = []

        for mode in range(4):
            #print("###### MODE "+str(mode)+" ########\n")
            MSTs = self.calculateMSTs(self.BinaryRobotRegions, self.droneNo, self.rows, self.cols, mode)
            AllRealPaths = []
            for r in range(self.droneNo):
                ct = CalculateTrajectories(self.rows, self.cols, MSTs[r])
                ct.initializeGraph(self.CalcRealBinaryReg(self.BinaryRobotRegions[r], self.rows, self.cols), True)
                ct.RemoveTheAppropriateEdges()
                ct.CalculatePathsSequence(4 * self.init_robot_pos[r][0] * self.cols + 2 * self.init_robot_pos[r][1])
                AllRealPaths.append(ct.PathSequence)

            # To print all the paths plannes
            #print(AllRealPaths)
            FinalPaths.append(AllRealPaths)
            #print("\n\n\n")

            TypesOfLines = np.zeros((self.rows*2, self.cols*2, 2))
            for r in range(self.droneNo):
                flag = False
                for connection in AllRealPaths[r]:
                    if flag:
                        if TypesOfLines[connection[0]][connection[1]][0] == 0:
                            indxadd1 = 0
                        else:
                            indxadd1 = 1

                        if TypesOfLines[connection[2]][connection[3]][0] == 0 and flag:
                            indxadd2 = 0
                        else:
                            indxadd2 = 1
                    else:
                        if not (TypesOfLines[connection[0]][connection[1]][0] == 0):
                            indxadd1 = 0
                        else:
                            indxadd1 = 1
                        if not (TypesOfLines[connection[2]][connection[3]][0] == 0 and flag):
                            indxadd2 = 0
                        else:
                            indxadd2 = 1

                    flag = True
                    if connection[0] == connection[2]:
                        if connection[1] > connection[3]:
                            TypesOfLines[connection[0]][connection[1]][indxadd1] = 2
                            TypesOfLines[connection[2]][connection[3]][indxadd2] = 3
                        else:
                            TypesOfLines[connection[0]][connection[1]][indxadd1] = 3
                            TypesOfLines[connection[2]][connection[3]][indxadd2] = 2

                    else:
                        if (connection[0] > connection[2]):
                            TypesOfLines[connection[0]][connection[1]][indxadd1] = 1
                            TypesOfLines[connection[2]][connection[3]][indxadd2] = 4
                        else:
                            TypesOfLines[connection[0]][connection[1]][indxadd1] = 4
                            TypesOfLines[connection[2]][connection[3]][indxadd2] = 1

            subCellsAssignment = np.zeros((2*self.rows, 2*self.cols))
            for i in range(self.rows):
                for j in range(self.cols):
                    subCellsAssignment[2 * i][2 * j] = self.A[i][j]
                    subCellsAssignment[2 * i + 1][2 * j] = self.A[i][j]
                    subCellsAssignment[2 * i][2 * j + 1] = self.A[i][j]
                    subCellsAssignment[2 * i + 1][2 * j + 1] = self.A[i][j]

            drone_turns = turns(AllRealPaths)
            drone_turns.count_turns()
            mode_to_drone_turns[mode] = drone_turns

            if self.visualization:
                image = visualize_paths(AllRealPaths, subCellsAssignment, self.droneNo, self.color)
                image.visualize_paths(mode)

        #print("\nResults:\n")

        for mode, val in mode_to_drone_turns.items():
            #printing initial positions of the bots in long form as CSV
            #for in_pos_input in pos_in_long:
            #    print(in_pos_input,end=",")
            
            #printing the mode for which the values are being printed
            
            
            straightlinemoves = []
            for i in range(self.droneNo):
                straightlinemoves.append(len(FinalPaths[mode][i])-mode_to_drone_turns[mode].turns[i])
                #print("Straight line moves for robot " + str(i) + " = " + str(len(FinalPaths[mode][i])-mode_to_drone_turns[mode].turns[i]))
            #print("Straight line moves: ",end="")

            #prints straight line moves as CSV
            #for num_straights in straightlinemoves:
                #print(",",num_straights,sep='',end="")
            
            #print straight line moves as a list
            #print(straightlinemoves, end="   ")

            #print(val)   #val is an Object that stores data about the turns. : val.turns , val.avg (average turns), val.std (std dev of turns)

            #printing only number of turns
            #print("Turns: ",end="")

            #prints values of number of turns as CSV
            #for num_turns in val.turns:
                #print(",",num_turns,sep='',end="")
            
            #prints value of turns as a list
            #print(val.turns, end="   ")

            #print("")
            #self.final_local = []
            final_local = {'NA':0}
            final_local['Initial Positions'] = pos_in_long
            final_local['Mode'] = str(mode)
            final_local['Straight Moves'] = straightlinemoves
            final_local['Turns'] = val.turns
            final_global.append(final_local)


    def CalcRealBinaryReg(self, BinaryRobotRegion, rows, cols):
        temp = np.zeros((2*rows, 2*cols))
        RealBinaryRobotRegion = np.zeros((2 * rows, 2 * cols), dtype=bool)
        for i in range(2*rows):
            for j in range(2*cols):
                temp[i, j] = BinaryRobotRegion[(int(i / 2))][(int(j / 2))]
                if temp[i, j] == 0:
                    RealBinaryRobotRegion[i, j] = False
                else:
                    RealBinaryRobotRegion[i, j] = True

        return RealBinaryRobotRegion

    def calculateMSTs(self, BinaryRobotRegions, droneNo, rows, cols, mode):
        MSTs = []
        for r in range(self.droneNo):
            k = Kruskal(rows, cols)
            k.initializeGraph(self.BinaryRobotRegions[r, :, :], True, mode)
            k.performKruskal()
            MSTs.append(k.mst)
        return MSTs

## The actual Main

In [12]:
random.seed(42)
input_file = open("drive/MyDrive/AIITRA/data.txt","r")
initial_pos = []
content = input_file.read()
map_wise = content.split("\n\n")


whole_map = [str(x) for x in range(2500)]
obstacle_list = []
obstacle_list.append([])

for x in range(len(map_wise)):
    obstacle_list.append(map_wise[x].split(" "))

def Diff(li1, li2):
    li_dif = [i for i in li1 if i not in li2]
    return li_dif

def grid_pos(long_pos):
    position = int(long_pos)
    rows = 50
    cols = 50
    [i,j]=[position % cols, position// cols]
    return i,j

map_number = 3 #Range from 0 to 3 
robot_possible_locations = Diff(whole_map,obstacle_list[map_number])
bot_count = 5
count = 0

while count < 1000:
    count += 1
    pos = []
    loc = []
    dist = []
    for bot in range(bot_count):
      x = int(random.choice(robot_possible_locations))
      loc.append(grid_pos(x))
      pos.append(x)

    for x1,y1 in loc:
      for x2, y2 in loc:
          if x1!=x2 and y1!=y2:
            dist.append(distance.euclidean((x1,y1), (x2,y2)))
    
    # obstacles = ' '.join(obstacle_list[map_number])
    # command = ""
    # command += ("python3 darpinPoly.py -grid 50 50 -in_pos " + positions + " -nep -portions ")
    # #print("python3 darpinPoly.py -grid 50 50 -in_pos " + positions,end=" -nep -portions ")
    # for bot in range(bot_count):
    #     command += (str(1/bot_count)+" ")
    # command += ("-obs_pos "+obstacles)
    # output_file.write(command+"\n")

    if min(dist) < 15:
        count -=1
        continue
    initial_pos.append(pos)

In [13]:
initial_pos[0]

[2442, 1139, 26, 653, 1751]

In [14]:
#HyperParams
rows, cols = 50, 50
nep = True
obs_pos = [x for x in range(1250,1270)] + [x for x in range(2029,2479+50,50)]
port = [0.2, 0.2, 0.2, 0.2, 0.2]
vis = False

MaxIter = 1024
CCvariation = 0.01
randomLevel = 0.0001
dcells = 2
importance = False

final_global = []

In [15]:
from tqdm.notebook import trange, tqdm

## The actual actual Main

In [16]:
def plan_path(checkpoint, next=True):
  global pos_in_long, final_global
  
  if next and os.path.exists(os.path.join(checkpoint, "final_file.csv")):
    final_df = pd.read_csv(checkpoint + "final_file.csv")

    with open(checkpoint + 'epoch.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
      x = pickle.load(f)
  
  else:
    final_df = pd.DataFrame({})
    x=0
      
  for m in trange(x, 1000):
      
      in_pos = initial_pos[m]
      obstacles_positions = []
      initial_positions = []

      # finding row and column of bot position from grid position
      for position in in_pos:
          if position < 0 or position >= rows*cols:
              print("Initial positions should be inside the Grid.")
              sys.exit(2)
          initial_positions.append((position // cols, position % cols))
          #saving position input to print for path optimization simulation results
          
          pos_in_long = in_pos

      # finding row and column of obstalces position from grid position
      for obstacle in obs_pos:
          if obstacle < 0 or obstacle >= rows*cols:
              print("Obstacles should be inside the Grid.")
              sys.exit(3)
          obstacles_positions.append((obstacle // cols, obstacle % cols))

      portions = []
      if nep:
          for portion in port:
              portions.append(portion)
      else:
          for bot in range(len(initial_positions)):
              portions.append(1/len(initial_positions))

      if len(initial_positions) != len(portions):
          print("Portions should be defined for each Bot")
          sys.exit(4)

      s = sum(portions)
      if abs(s-1) >= 0.0001:
          print("Sum of portions should be equal to 1.")
          sys.exit(1)

      for position in initial_positions:
          for obstacle in obstacles_positions:
              if position[0] == obstacle[0] and position[1] == obstacle[1]:
                  print("Initial positions should not be on obstacles")
                  sys.exit(3)




      # Printing initial conditions
      #print("\nInitial Conditions Defined:")
      #print("Grid Dimensions:", rows, cols)
      #print("No of bots :", len(initial_positions))
      #print("Initial Robots' positions", initial_positions)
      #print("Portions for each Robot:", portions, "\n")

      poly = DARPinPoly(rows, cols, MaxIter, CCvariation, randomLevel, dcells, importance, nep, initial_positions, portions, obstacles_positions, vis)
      final_global = pd.DataFrame(final_global)
      final_df = pd.concat([final_df,final_global])
      final_global = []
      
      with open(checkpoint + 'epoch.pkl', 'wb') as f:  
        pickle.dump(m, f)
      final_df.to_csv(checkpoint+'final_file.csv', index=False)


In [ ]:
pos_in_long = []
checkpoint = "drive/MyDrive/AIITRA/"
plan_path(checkpoint)


  0%|          | 0/537 [00:00<?, ?it/s]